In [52]:
import numpy as np
import matplotlib.pyplot as plt
import torch as t
import torch.nn as nn
import torchvision as tv
import os

In [64]:
def dynamic_binarization(img):
    return t.distributions.bernoulli.Bernoulli(img).sample()

In [114]:
path = '/T480/AnacondaProjects/svgd/'
train_data = tv.datasets.MNIST(path,train = True,download = True, 
                               transform = tv.transforms.Compose(
                                   [tv.transforms.ToTensor(),tv.transforms.Lambda(dynamic_binarization)]))
test_data = tv.datasets.MNIST(path,train = False, download = True, transform = tv.transforms.Compose(
                                   [tv.transforms.ToTensor(),tv.transforms.Lambda(dynamic_binarization)]))
train_loader = t.utils.data.DataLoader(train_data,batch_size = batch_size, shuffle = True)
test_loader = t.utils.data.DataLoader(test_data,batch_size = batch_size, shuffle = True)

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        